In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import metrics

In [3]:
cars = pd.read_csv('vehicles.csv')

In [159]:
cars

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,0,7240372487,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,35990,2010.0,chevrolet,corvette grand sport,good,...,rwd,NaN,other,NaN,https://images.craigslist.org/00N0N_ipkbHVZYf4...,Carvana is the safer way to buy a car During t...,al,32.590000,-85.480000,2020-12-02T08:11:30-0600
1,1,7240309422,https://auburn.craigslist.org/cto/d/auburn-201...,auburn,https://auburn.craigslist.org,7500,2014.0,hyundai,sonata,excellent,...,fwd,NaN,sedan,NaN,https://images.craigslist.org/00s0s_gBHYmJ5o7y...,I'll move to another city and try to sell my c...,al,32.547500,-85.468200,2020-12-02T02:11:50-0600
2,2,7240224296,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,4900,2006.0,bmw,x3 3.0i,good,...,NaN,NaN,SUV,blue,https://images.craigslist.org/00B0B_5zgEGWPOrt...,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al,32.616807,-85.464149,2020-12-01T19:50:41-0600
3,3,7240103965,https://auburn.craigslist.org/cto/d/lanett-tru...,auburn,https://auburn.craigslist.org,2000,1974.0,chevrolet,c-10,good,...,rwd,full-size,pickup,blue,https://images.craigslist.org/00M0M_6o7KcDpArw...,1974 chev. truck (LONG BED) NEW starter front ...,al,32.861600,-85.216100,2020-12-01T15:54:45-0600
4,4,7239983776,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,19500,2005.0,ford,f350 lariat,excellent,...,4wd,full-size,pickup,blue,https://images.craigslist.org/00p0p_b95l1EgUfl...,2005 Ford F350 Lariat (Bullet Proofed). This t...,al,32.547500,-85.468200,2020-12-01T12:53:56-0600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458208,458208,7240959076,https://wausau.craigslist.org/ctd/d/wisconsin-...,wausau,https://wausau.craigslist.org,14995,2014.0,cadillac,srx,excellent,...,4wd,mid-size,SUV,silver,https://images.craigslist.org/00808_1bVcaYZtCy...,2014 Cadillac SRX All Wheel Drive Luxury Colle...,wi,44.408867,-89.881210,2020-12-03T08:56:29-0600
458209,458209,7240954682,https://wausau.craigslist.org/ctd/d/wisconsin-...,wausau,https://wausau.craigslist.org,17895,2018.0,hyundai,tucson,excellent,...,4wd,mid-size,SUV,white,https://images.craigslist.org/00l0l_gikkP11dEF...,2018 Hyundai Tucson SE 2.0 I4 AWD. Carfax ver...,wi,44.408867,-89.882240,2020-12-03T08:48:08-0600
458210,458210,7240645700,https://wyoming.craigslist.org/cto/d/sheridan-...,wyoming,https://wyoming.craigslist.org,17900,2014.0,cadillac,cts coupe awd,excellent,...,4wd,mid-size,coupe,black,https://images.craigslist.org/00t0t_lNnW4cboOG...,"53,000 miles. Cosmetically, in near mint condi...",wy,44.784900,-106.964800,2020-12-02T14:09:45-0700
458211,458211,7240600465,https://wyoming.craigslist.org/cto/d/sheridan-...,wyoming,https://wyoming.craigslist.org,1300,2008.0,jeep,grand cherokee,good,...,4wd,mid-size,SUV,white,https://images.craigslist.org/00C0C_fl0NW1IeJw...,PRICE REDUCTION Turns out the engine is toast...,wy,44.773500,-106.939600,2020-12-02T13:01:04-0700


In [4]:
cars_naless = cars.drop(columns=['Unnamed: 0', 'url', 'region_url', 'VIN', 'image_url', 'lat','long','posting_date']).dropna()

In [5]:
cars_0_naless = cars_naless[cars_naless['price'] != 0].reset_index(drop=True)

In [8]:
cleaned_cars = cars_0_naless[(cars_0_naless['price'] < 300000) & (cars_0_naless['price'] > 100)]
cleaned_cars = cleaned_cars[cleaned_cars['odometer'] < 500000]

In [9]:
desc_counts = cleaned_cars['description'].value_counts()
desc_dict = desc_counts[desc_counts > 1].to_dict()
cleaned_cars['duplicate_descriptions'] = cleaned_cars['description'].apply(lambda x: desc_dict.get(x, 0))

In [10]:
year_max = cleaned_cars['year'].max()
year_min = cleaned_cars['year'].min()
((cleaned_cars['year'] - year_min)/(year_max - year_min)).tolist()

[0.5,
 0.8297872340425532,
 0.8085106382978723,
 0.9893617021276596,
 0.8829787234042553,
 0.9361702127659575,
 0.9468085106382979,
 0.9468085106382979,
 0.8297872340425532,
 0.9042553191489362,
 0.9468085106382979,
 0.925531914893617,
 0.9468085106382979,
 0.8404255319148937,
 0.8723404255319149,
 0.9574468085106383,
 0.9893617021276596,
 0.9787234042553191,
 0.9361702127659575,
 0.925531914893617,
 0.8936170212765957,
 0.9361702127659575,
 0.8617021276595744,
 0.9468085106382979,
 0.9042553191489362,
 0.9468085106382979,
 0.9042553191489362,
 0.9361702127659575,
 0.8404255319148937,
 0.851063829787234,
 0.8404255319148937,
 0.925531914893617,
 0.925531914893617,
 0.9574468085106383,
 0.8404255319148937,
 0.9468085106382979,
 0.9680851063829787,
 0.9787234042553191,
 0.9468085106382979,
 0.9893617021276596,
 0.9680851063829787,
 0.6914893617021277,
 0.8085106382978723,
 0.43617021276595747,
 0.9468085106382979,
 0.8829787234042553,
 0.9787234042553191,
 0.8829787234042553,
 0.85106382

In [11]:
# Features Transforming
# Numerical
year = ((cleaned_cars['year'] - year_min)/(year_max - year_min)).tolist()
odom = (cleaned_cars['odometer']/max(cleaned_cars['odometer'])).tolist()

# Categorical/One Hot Encode
manu = cleaned_cars['manufacturer'].unique().tolist()
cond = cleaned_cars['condition'].unique().tolist()
cycl = cleaned_cars['cylinders'].unique().tolist()
fuel = cleaned_cars['fuel'].unique().tolist()
tit = cleaned_cars['title_status'].unique().tolist()
tran = cleaned_cars['transmission'].unique().tolist()
drive = cleaned_cars['drive'].unique().tolist()
size = cleaned_cars['size'].unique().tolist()
typ = cleaned_cars['type'].unique().tolist()
paint = cleaned_cars['paint_color'].unique().tolist()
state = cleaned_cars['state'].unique().tolist()
dupl = cleaned_cars['duplicate_descriptions'].unique().tolist()

In [12]:
# One Hot vector for each feature
manu_One = np.zeros((len(cleaned_cars),len(manu)))
cond_One = np.zeros((len(cleaned_cars),len(cond)))
cycl_One = np.zeros((len(cleaned_cars),len(cycl)))
fuel_One = np.zeros((len(cleaned_cars),len(fuel)))
tit_One = np.zeros((len(cleaned_cars),len(tit)))
tran_One = np.zeros((len(cleaned_cars),len(tran)))
drive_One = np.zeros((len(cleaned_cars),len(drive)))
size_One = np.zeros((len(cleaned_cars),len(size)))
typ_One = np.zeros((len(cleaned_cars),len(typ)))
paint_One = np.zeros((len(cleaned_cars),len(paint)))
state_One = np.zeros((len(cleaned_cars),len(state)))
dupl_One = np.zeros((len(cleaned_cars),len(dupl)))

In [13]:
# Transform all features
for x in range(len(cleaned_cars)):
    manu_One[x][manu.index(cleaned_cars['manufacturer'].iloc[x])] = 1
    cond_One[x][cond.index(cleaned_cars['condition'].iloc[x])] = 1
    cycl_One[x][cycl.index(cleaned_cars['cylinders'].iloc[x])] = 1
    fuel_One[x][fuel.index(cleaned_cars['fuel'].iloc[x])] = 1
    tit_One[x][tit.index(cleaned_cars['title_status'].iloc[x])] = 1
    tran_One[x][tran.index(cleaned_cars['transmission'].iloc[x])] = 1
    drive_One[x][drive.index(cleaned_cars['drive'].iloc[x])] = 1
    size_One[x][size.index(cleaned_cars['size'].iloc[x])] = 1
    typ_One[x][typ.index(cleaned_cars['type'].iloc[x])] = 1
    paint_One[x][paint.index(cleaned_cars['paint_color'].iloc[x])] = 1
    state_One[x][state.index(cleaned_cars['state'].iloc[x])] = 1
    dupl_One[x][dupl.index(cleaned_cars['duplicate_descriptions'].iloc[x])] = 1

In [72]:
# Get model category counts and transform them for those above 100
categoryCounts = {}
for x in cleaned_cars['model']:
    if x not in categoryCounts.keys():
        categoryCounts[x] = 0
    categoryCounts[x] += 1
categories = [c for c in categoryCounts if categoryCounts[c] > 100]
catID = dict(zip(list(categories),range(len(categories))))

mod_One = np.zeros((len(cleaned_cars),len(catID.keys())))
for x in range(len(cleaned_cars)):
    if cleaned_cars['model'].iloc[x] in catID.keys():
        mod_One[x][catID[cleaned_cars['model'].iloc[x]]] = 1

In [78]:
# List of all features to append
car_columns = [year,
               odom,
               manu_One,
               mod_One,
               cond_One,
               cycl_One,
               fuel_One,
               tit_One,
               tran_One,
               drive_One,
               size_One,
               typ_One,
               paint_One,
               state_One]

# array to put all features in
clean_car_data = np.array([])

for i in car_columns:
    if clean_car_data.size == 0:
        clean_car_data = np.array([i]).reshape(-1, 1)
    else:
        data = np.array(i)
        
        if len(data.shape) == 1:
            data = data.reshape(-1, 1)
            
        clean_car_data = np.concatenate((clean_car_data, data), axis=1)

In [115]:
# Get training data
x_train, x_test, y_train, y_test = train_test_split(clean_car_data, cleaned_cars['price'].values, test_size=0.3, random_state=42)

In [116]:
# Get validation and testing data
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [117]:
def RMSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return np.sqrt(sum(differences) / len(differences))

In [118]:
# Make and train linear regression model
lin_reg_model = LinearRegression(fit_intercept=True)
lin_reg_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [146]:
# Prediction on train
lin_reg_train_preds = lin_reg_model.predict(x_train)
lin_reg_train_RMSE = RMSE(lin_reg_train_preds, y_train)
lin_reg_train_R2 = lin_reg_model.score(x_train, y_train)
lin_reg_train_RMSE, lin_reg_train_R2

(7038.20539095947, 0.6082326250860152)

In [145]:
# Prediction on validation
lin_reg_valid_preds = lin_reg_model.predict(x_valid)
lin_reg_valid_RMSE = RMSE(lin_reg_valid_preds, y_valid)
lin_reg_valid_R2 = lin_reg_model.score(x_valid, y_valid)
lin_reg_valid_RMSE, lin_reg_valid_R2

(7063.588245218276, 0.6052094854542602)

In [158]:
# Prediction on validation
lin_reg_test_preds = lin_reg_model.predict(x_test)
lin_reg_test_RMSE = RMSE(lin_reg_test_preds, y_test)
lin_reg_test_R2 = lin_reg_model.score(x_test, y_test)
lin_reg_test_RMSE, lin_reg_test_R2

(7178.25523905023, 0.6139248520251501)

### Ablation

In [ ]:
def create_features(columns_list):
    """Creates all features to put into a features list."""

    # array to put all features in
    clean_car_data = np.array([])

    for i in columns_list:
        if clean_car_data.size == 0:
            data = np.array(i)
            
            if len(data.shape) == 1:
                data = data.reshape(-1, 1)
                
            clean_car_data = data
        else:
            data = np.array(i)
            
            if len(data.shape) == 1:
                data = data.reshape(-1, 1)

            clean_car_data = np.concatenate((clean_car_data, data), axis=1)

    return clean_car_data

def get_evaluations(x_train, x_valid, y_train, y_valid):
    """Train and test using data"""
    # Make and train linear regression model
    model = LinearRegression(fit_intercept=True)
    model.fit(x_train, y_train)
    
    # Predict on train
    train_preds = model.predict(x_train)
    train_RMSE = RMSE(train_preds, y_train)
    
    # Predict on validation
    valid_preds = model.predict(x_valid)
    valid_RMSE = RMSE(valid_preds, y_valid)
    
    return model, train_RMSE, valid_RMSE
    
def find_ablation(columns_list, y):
    """Find the best columns through ablation."""
    
    all_models = {}

    all_features = create_features(columns_list)

    x_tr, x_te, y_tr, y_te = train_test_split(all_features, y, test_size=0.3, random_state=42)
    x_va, x_te, y_va, y_te = train_test_split(x_te, y_te, test_size=0.5, random_state=42)

    temp_model, tr_RMSE, va_RMSE = get_evaluations(x_tr, x_va, y_tr, y_va)

    all_models['all'] = temp_model
    print("With all features: Train RMSE " + str(tr_RMSE) + " | Valid RMSE " + str(va_RMSE))
    
    for i in range(len(columns_list)):
        modded_list = columns_list.copy()
        modded_list.pop(i)
        
        all_features = create_features(modded_list)
        
        x_tr, x_te, y_tr, y_te = train_test_split(all_features, y, test_size=0.3, random_state=42)
        x_va, x_te, y_va, y_te = train_test_split(x_te, y_te, test_size=0.5, random_state=42)
        
        temp_model, tr_RMSE, va_RMSE = get_evaluations(x_tr, x_va, y_tr, y_va)
        
        all_models[i] = temp_model
        print("Without feature " + str(i) + ": Train RMSE " + str(tr_RMSE) + " | Valid RMSE " + str(va_RMSE))
    
    return all_models

def find_double_ablation(columns_list, y):
    """Find the best columns through ablation."""
    
    all_models = {}

    all_features = create_features(columns_list)

    x_tr, x_te, y_tr, y_te = train_test_split(all_features, y, test_size=0.3, random_state=42)
    x_va, x_te, y_va, y_te = train_test_split(x_te, y_te, test_size=0.5, random_state=42)

    temp_model, tr_RMSE, va_RMSE = get_evaluations(x_tr, x_va, y_tr, y_va)

    all_models['all'] = temp_model
    print("With all features: Train RMSE " + str(tr_RMSE) + " | Valid RMSE " + str(va_RMSE))
    
    for i in range(len(columns_list)):
        for j in range(i, len(columns_list) - 1):
            modded_list = columns_list.copy()
            modded_list.pop(i)
            modded_list.pop(j)

            all_features = create_features(modded_list)

            x_tr, x_te, y_tr, y_te = train_test_split(all_features, y, test_size=0.3, random_state=42)
            x_va, x_te, y_va, y_te = train_test_split(x_te, y_te, test_size=0.5, random_state=42)

            temp_model, tr_RMSE, va_RMSE = get_evaluations(x_tr, x_va, y_tr, y_va)

            all_models[(i, j)] = temp_model
            print("Without features " + str(i) + " and " + str(j+1) +
                  ": Train RMSE " + str(tr_RMSE) + " | Valid RMSE " + str(va_RMSE))

    return all_models

In [40]:
first_ablation_columns = [year,
               odom,
               manu_One,
               mod_One,
               cond_One,
               cycl_One,
               fuel_One,
               tit_One,
               tran_One,
               drive_One,
               size_One,
               typ_One,
               paint_One,
               state_One]

In [41]:
second_ablation_columns = [year,
               odom,
               manu_One,
               mod_One,
               cond_One,
               cycl_One,
               fuel_One,
               tit_One,
               tran_One,
               drive_One,
               size_One,
               typ_One,
               paint_One,
               state_One,
               dupl_One]

In [42]:
first_ablation = find_ablation(first_ablation_columns, cleaned_cars['price'])

With all features: Train RMSE 7038.20539095947 | Valid RMSE 7063.588245218276
Without feature 0: Train RMSE 7361.644245886838 | Valid RMSE 7414.553759997701
Without feature 1: Train RMSE 8061.683367647537 | Valid RMSE 8097.260633503978
Without feature 2: Train RMSE 7196.199959210021 | Valid RMSE 7239.594549727369
Without feature 3: Train RMSE 7177.791922205131 | Valid RMSE 7172.645125939519
Without feature 4: Train RMSE 7111.346500799724 | Valid RMSE 7121.036221717426
Without feature 5: Train RMSE 7109.996229638093 | Valid RMSE 7150.663267817337
Without feature 6: Train RMSE 7215.939827173551 | Valid RMSE 7270.543239621116
Without feature 7: Train RMSE 7074.384771850013 | Valid RMSE 7090.701408975808
Without feature 8: Train RMSE 7049.017571476253 | Valid RMSE 7081.074770555492
Without feature 9: Train RMSE 7109.2379208525435 | Valid RMSE 7158.770301301005
Without feature 10: Train RMSE 7045.3460556299015 | Valid RMSE 7070.366200328121
Without feature 11: Train RMSE 7140.935397058791 |

In [43]:
second_ablation = find_ablation(second_ablation_columns, cleaned_cars['price'])

With all features: Train RMSE 7015.787949824971 | Valid RMSE 7040.715978382766
Without feature 0: Train RMSE 7338.953497028758 | Valid RMSE 7390.685293527036
Without feature 1: Train RMSE 8036.3505485306805 | Valid RMSE 8075.6584090428305
Without feature 2: Train RMSE 7171.987393401761 | Valid RMSE 7214.108937455506
Without feature 3: Train RMSE 7149.51298351505 | Valid RMSE 7142.7052737143395
Without feature 4: Train RMSE 7090.248243313105 | Valid RMSE 7100.451708194844
Without feature 5: Train RMSE 7087.515385114727 | Valid RMSE 7127.5662135292505
Without feature 6: Train RMSE 7191.743999128722 | Valid RMSE 7244.369975624002
Without feature 7: Train RMSE 7052.201782494228 | Valid RMSE 7068.706618621503
Without feature 8: Train RMSE 7026.696960675629 | Valid RMSE 7058.410631306735
Without feature 9: Train RMSE 7087.008764843684 | Valid RMSE 7134.793232234086
Without feature 10: Train RMSE 7023.585908058825 | Valid RMSE 7048.5934851906595
Without feature 11: Train RMSE 7117.86179442041

In [60]:
first_double_ablation = find_double_ablation(first_ablation_columns, cleaned_cars['price'])

With all features: Train RMSE 7038.197748807188 | Valid RMSE 7063.529033458708
Without features 0 and 1: Train RMSE 8602.039307045397 | Valid RMSE 8674.089117351383
Without features 0 and 2: Train RMSE 7512.598036257236 | Valid RMSE 7589.941954618639
Without features 0 and 3: Train RMSE 7530.600317120363 | Valid RMSE 7547.057206575836
Without features 0 and 4: Train RMSE 7505.756023759494 | Valid RMSE 7539.260005666788
Without features 0 and 5: Train RMSE 7392.182994826418 | Valid RMSE 7457.166521293509
Without features 0 and 6: Train RMSE 7540.623222235628 | Valid RMSE 7632.2178313696395
Without features 0 and 7: Train RMSE 7396.962609406127 | Valid RMSE 7437.668877879769
Without features 0 and 8: Train RMSE 7362.019735305869 | Valid RMSE 7415.20785529813
Without features 0 and 9: Train RMSE 7455.381683165372 | Valid RMSE 7535.194052828997
Without features 0 and 10: Train RMSE 7370.086937810868 | Valid RMSE 7418.874526859306
Without features 0 and 11: Train RMSE 7455.3531264745525 | V

In [35]:
limit_5_columns = [year,
               odom,
               manu_One,
               mod_One,
               cond_One,
               cycl_One,
               fuel_One,
               tit_One,
               tran_One,
               drive_One,
               size_One,
               typ_One,
               paint_One,
               state_One]

In [28]:
# Above 50 appearances for model feature
limit_50_features = create_features(limit_50_columns)
        
x_tr_50, x_te_50, y_tr_50, y_te_50 = train_test_split(limit_50_features, cleaned_cars['price'], test_size=0.3, random_state=42)
x_va_50, x_te_50, y_va_50, y_te_50 = train_test_split(x_te_50, y_te_50, test_size=0.5, random_state=42)
        
model_50, train_50_RMSE, valid_50_RMSE = get_evaluations(x_tr_50, x_va_50, y_tr_50, y_va_50)

print("Train RMSE " + str(train_50_RMSE))
print("Valid RMSE " + str(valid_50_RMSE))

Train RMSE 6985.143242854444
Valid RMSE 7027.023435321906


In [32]:
# Above 25 appearances for model feature
limit_25_features = create_features(limit_25_columns)
        
x_tr_25, x_te_25, y_tr_25, y_te_25 = train_test_split(limit_25_features, cleaned_cars['price'], test_size=0.3, random_state=42)
x_va_25, x_te_25, y_va_25, y_te_25 = train_test_split(x_te_25, y_te_25, test_size=0.5, random_state=42)
        
model_25, train_25_RMSE, valid_25_RMSE = get_evaluations(x_tr_25, x_va_25, y_tr_25, y_va_25)

print("Train RMSE " + str(train_25_RMSE))
print("Valid RMSE " + str(valid_25_RMSE))

Train RMSE 6899.385718610281
Valid RMSE 6971.069484939485


In [156]:
# Above 5 appearances for model feature
limit_5_features = create_features(limit_5_columns)
        
x_tr_5, x_te_5, y_tr_5, y_te_5 = train_test_split(limit_5_features, cleaned_cars['price'], test_size=0.3, random_state=42)
x_va_5, x_te_5, y_va_5, y_te_5 = train_test_split(x_te_5, y_te_5, test_size=0.5, random_state=42)
        
model_5, train_5_RMSE, valid_5_RMSE = get_evaluations(x_tr_5, x_va_5, y_tr_5, y_va_5)

model_5_test_preds = model_5.predict(x_te_5)
test_5_RMSE = RMSE(model_5_test_preds, y_te_5)

print("Train RMSE " + str(train_5_RMSE))
print("Valid RMSE " + str(valid_5_RMSE))
print("Test RMSE " + str(test_5_RMSE))

Train RMSE 6475.597962947909
Valid RMSE 6756.026734052901
Test RMSE 6997.280452253276


In [157]:
train_5_R2 = model_5.score(x_tr_5, y_tr_5)
valid_5_R2 = model_5.score(x_va_5, y_va_5)
test_5_R2 = model_5.score(x_te_5, y_te_5)

print("Train R2 Score " + str(train_5_R2))
print("Valid R2 Score " + str(valid_5_R2))
print("Test R2 Score " + str(test_5_R2))

Train R2 Score 0.6683621054294325
Valid R2 Score 0.6388408035591318
Test R2 Score 0.6331465437077597


### Only Average Baseline

In [163]:
only_average = [np.mean(cleaned_cars['price'])]*len(cleaned_cars['price'])
only_average

[12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015545532,
 12547.328015

In [168]:
print("Baseline RMSE: " + str(RMSE(only_average, cleaned_cars['price'].values)))
print("Baseline R2 Score: " + str(metrics.r2_score(only_average, cleaned_cars['price'].values)))

Baseline RMSE: 11291.090641925419
Baseline R2 Score: -3.853110371225134e+31
